In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from gurobipy import *
import osmnx as ox
ox.config(use_cache=True, log_console=True)
from itertools import permutations

In [2]:
# Number of Candidate DCs chosen from Facility Location Model
DC_N = 1

# List Creator for DCs
def createList_DC(r1, r2):
    return list(['DC%d'%x for x in range(r1, r2+1)])

#create an index
DC_ID = createList_DC(1, DC_N)

#Gi = fixed cost for establishing depot i
fixed_cost_DC = [1500] * DC_N 

#Maximum Throughput at depot i = Vi
capacity_DC = [250] * DC_N

#Variable Warehousing Cost (Picking)
varCost_DC = [0.5, 0.4, 0.2, 0.8]

#Position of the DCs
lat_DC = [49.785592, 49.773391, 49.800460, 49.811422, 49.791321]
lon_DC = [9.930666, 9.923226, 9.951607, 9.981127, 9.953342]

# Osmid of the Dcs
osmid = [27489160, 30747843, 40430241, 27352149, 27457548]

# Time Windows ai and bi
window_start = [0] * DC_N
window_end = [600] *DC_N

dc_tuples = list(zip(DC_ID, fixed_cost_DC, capacity_DC, varCost_DC, lat_DC, lon_DC, osmid, window_start, window_end))

set_of_all_DC = pd.DataFrame(dc_tuples, columns = ["DC_ID", "fixed_cost_DC", "capacity_DC", "varCost_DC", "lat", "lon", "osmid", "window_start","window_end"])
set_of_all_DC.set_index("DC_ID", inplace = True)

# Define Index I for model building
I = set_of_all_DC.index.values

# DC_S
DC_S = set_of_all_DC["osmid"].values.tolist()

set_of_all_DC

,fixed_cost_DC,capacity_DC,varCost_DC,lat,lon,osmid,window_start,window_end
DC_ID,,,,,,,,
DC1,1500,250,0.5,49.785592,9.930666,27489160,0,600


In [3]:
# Sample Customers from osmnx

# Define Customer N and Demand per
customers = 2
demand_per_customer = 5
service_time = 5

# Function to create a Customer Index List
def createList_C(r1, r2):
    return list(['C%d'%x for x in range(r1, r2+1)])

# Get all nodes in Würzburg from OSMNX which will be used to sample a certain amount of customers
G = ox.graph_from_place("Wuerzburg, Germany", network_type = "drive")
Gs = ox.utils_graph.get_largest_component(G, strongly = True)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(Gs)

# Create nodes_df which is the basis for our customer df
nodes_df = gdf_nodes[["y", "x"]].copy()
nodes_df.columns = ["lat", "lon"]

#Sample from nodes_df
sample_nodes_df = nodes_df.sample(n = customers, random_state= 3)


#DF Manipulation
C_ID = createList_C(1,customers)
osmid = list(sample_nodes_df.index.values)
sample_nodes_df = sample_nodes_df.rename(index=dict(zip(osmid,C_ID)))
sample_nodes_df.index.name = "C_ID"
sample_nodes_df["osmid"] = osmid

#Create final demand column for customer DF
mylist = [demand_per_customer] * customers
# service time
s_time = [service_time] * customers
# Time Windows
window_start = [0,0]#,0,0,0] #[50, 120, 200, 300, 450]
window_end = [600,600]#,600,600,600] #[100, 180, 280, 380, 550]
set_of_all_customers = sample_nodes_df.copy()
# Add Demand column from the newly created list
set_of_all_customers['Demand_C'] = mylist
set_of_all_customers["Service_Time"] = s_time
set_of_all_customers["window_start"] = window_start
set_of_all_customers["window_end"] = window_end
# Sort the df by osmid, this needs to be done for renaming purposes in the dist_matrix
set_of_all_customers = set_of_all_customers.sort_values(by=["osmid"])
# New index C1-C2 etc.
set_of_all_customers.reset_index(drop=True, inplace=True)
set_of_all_customers.index = C_ID
set_of_all_customers.index.names = ["C_ID"]
# Create Nodes_S for joining and plotting later
Nodes_S = set_of_all_customers["osmid"].tolist()
# Create index J for model building
J = set_of_all_customers.index.values
set_of_all_customers.head()

,lat,lon,osmid,Demand_C,Service_Time,window_start,window_end
C_ID,,,,,,,
C1,49.776658,9.952296,40932452,5,5,0,600
C2,49.791096,9.963971,6034416208,5,5,0,600


In [4]:
Nodes_S = set_of_all_customers["osmid"].tolist()
Nodes_S1 = DC_S + Nodes_S
# By combining the lists in this way, the DCs are first and customers second
#Nodes_S1

In [5]:
# Construct set of all vehicles K

#Vehicle Count
V_N = 3

# Vehicle List Creator
def createList_V(r1, r2):
    return list(['V_%d'%x for x in range(r1, r2+1)])

# create index
V_ID = createList_V(1, V_N)

# Vehicle Capacity Qk
capacity_V = [100] * V_N

# fixed cost of using Vehicle Fk
fixed_cost_V = [5] * V_N

v_tuples = list(zip(V_ID, capacity_V, fixed_cost_V))

set_of_all_vehicles = pd.DataFrame(v_tuples, columns = ["V_ID", "capacity_V", "fixed_cost_V"])
set_of_all_vehicles.set_index("V_ID", inplace=True)
K = set_of_all_vehicles.index.values
set_of_all_vehicles

,capacity_V,fixed_cost_V
V_ID,,
V_1,100,5
V_2,100,5
V_3,100,5


In [6]:
# Add a Time matrix so that we can incorporate traveling times into our routes and cost calculation
F = ox.graph_from_place("Wuerzburg, Germany", network_type = "drive")
Fs = ox.utils_graph.get_largest_component(F, strongly = True)
Fs = ox.add_edge_speeds(Fs)
Fs = ox.add_edge_travel_times(Fs)
#by specifying weight = travel_time, the output is travel time between the nodes in seconds
mat_generator1 = nx.all_pairs_dijkstra_path_length(Fs, weight = "travel_time")
mat_dict1 = dict(mat_generator1)
mat1 = pd.DataFrame(mat_dict1).round(1)
mat1 = mat1.rename_axis("osmid").sort_values(by = ["osmid"])
mat1.head()

,10799058,10799066,10799083,10799085,10799087,10799937,10799942,10799945,10799947,10799949,...,8684424129,8917651907,8917667635,8917667643,8917704467,8917704493,8917704495,8917704496,8917704497,8981470540
osmid,,,,,,,,,,,,,,,,,,,,,
10799058,0.0,240.0,6.1,35.3,21.3,157.7,188.8,183.1,182.1,184.2,...,444.8,648.5,660.8,633.9,429.1,432.2,431.5,432.1,433.0,362.8
10799066,28.4,0.0,21.3,16.7,36.5,172.9,204.0,198.3,197.3,199.4,...,460.0,663.7,676.0,649.1,444.3,447.4,446.7,447.3,448.2,378.0
10799083,40.9,233.9,0.0,29.2,15.2,151.6,182.7,177.0,176.0,178.1,...,438.7,642.4,654.7,627.8,423.0,426.1,425.4,426.0,426.9,356.7
10799085,11.7,238.5,4.6,0.0,19.8,156.2,187.3,181.6,180.6,182.7,...,443.3,647.0,659.3,632.4,427.6,430.7,430.0,430.6,431.5,361.3
10799087,247.1,218.7,240.0,235.4,0.0,136.4,167.5,161.8,160.8,162.9,...,423.5,627.2,639.5,612.6,407.8,410.9,410.2,410.8,411.7,341.5


In [7]:
# Join Travel Times for the sampled nodes from our Time Matrix for all nodes 
time_matrix_t = pd.DataFrame(index=Nodes_S1)
# Rename the index for the join with the time matrix
time_matrix_t = time_matrix_t.rename_axis("osmid")
# Join new empty time_matrix with filled matrix for every chosen node
time_matrix_t = time_matrix_t.merge(mat1, left_index=True, right_index=True)
time_matrix_t = time_matrix_t[time_matrix_t.columns.intersection(Nodes_S1)]
#Rename Columns to fit the order of Nodes_S1 = DCs first then Customers. Index and columns have to match in position
time_matrix_t = time_matrix_t.reindex(columns=Nodes_S1)

nn = np.concatenate((I,J), axis=None)
new_colnames = nn.tolist()
time_matrix_t.columns = new_colnames
time_matrix_t.index = new_colnames
time_matrix_t.head()

,DC1,C1,C2
DC1,0.0,209.7,227.7
C1,240.2,0.0,262.1
C2,228.6,225.1,0.0


In [8]:
time_matrix = time_matrix_t * 0.0042
time_matrix.head()

,DC1,C1,C2
DC1,0.00000,0.88074,0.95634
C1,1.00884,0.00000,1.10082
C2,0.96012,0.94542,0.00000


In [9]:
time_mat_min = (time_matrix_t/60).round(1)
time_mat_min

,DC1,C1,C2
DC1,0.0,3.5,3.8
C1,4.0,0.0,4.4
C2,3.8,3.8,0.0


In [10]:
m = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-11-24
Using license file C:\Users\michi\gurobi.lic


In [11]:
# Routes
x = m.addVars([*I,*J],[*I,*J], K, name = "x", vtype=GRB.BINARY)
# Depot Open
y = m.addVars(I, name = "y", vtype =GRB.BINARY)
# Arrival Time of Delivery vehicle at customer j
aj = m.addVars([*I,*J], name = "aj", vtype= GRB.CONTINUOUS)
# Arrival Time of delivery vehicle at depot i
aik = m.addVars(I,K, name = "aik", vtype= GRB.CONTINUOUS)
# Waiting time of delivery vehicle at customer j
wdj = m.addVars([*I,*J], name = "wdj", vtype= GRB.CONTINUOUS)

In [12]:
M = 6000

In [13]:
fixedCost_depot = quicksum(y[i] * set_of_all_DC.loc[i].fixed_cost_DC for i in I)
variableCosts_transp = quicksum([time_matrix.loc[i,j] * x[i,j,k] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])
obj = fixedCost_depot + variableCosts_transp

In [14]:
m.setObjective(obj, GRB.MINIMIZE)

In [15]:
for i in [*I,*J]:
    for k in K:
        m.addConstr(quicksum(x[i,j,k] for j in [*I,*J] if i!=j) == quicksum(x[j,i,k] for j in [*I,*J] if i!=j))

In [16]:
for i in I:
    m.addConstr(quicksum(x[j,i,k] for j in [*I,*J] for k in K if i!=j) == 1)

In [17]:
for k in K:
    m.addConstr(quicksum(x[i,j,k] for i in I for j in J) <= 1)

In [18]:
for k in K:
    for i in I:
        m.addConstr(quicksum(x[i,j,k] for j in [*I,*J] if i!=j) <= y[i])

In [19]:
m.write("time.lp")

In [20]:
for k in K:
    m.addConstr(quicksum(set_of_all_customers.loc[j].Demand_C * x[i,j,k] for i in [*I,*J] for j in J if i!=j) <= set_of_all_vehicles.loc[k].capacity_V)

In [21]:
for i in [*I,*J]:
    for j in J:
        m.addConstr(aj[j] >= aj[i] + wdj[j] + time_mat_min.loc[i,j] + M*(quicksum(x[i,j,k] - 1 for k in K)))

In [22]:
for i in I:
    for j in J:
        for k in K:
            m.addConstr(aik[i,k] >= aj[j] + wdj[j] + time_mat_min.loc[i,j] + M * (x[i,j,k] -1))

In [23]:
for j in J:
    m.addConstr(aj[j] + wdj[j] <= set_of_all_customers.loc[j].window_start)

In [24]:
for j in J:
    m.addConstr(aj[j] + wdj[j] >= set_of_all_customers.loc[j].window_end)

In [25]:
for i in I:
    m.addConstr(aj[i] == 0)

In [26]:
for i in I:
    m.addConstr(wdj[i] == 0)

In [27]:
m.write("time.lp")

In [28]:
m.Params.LogFile = "GUR_TIME.log"
m.optimize()

Changed value of parameter LogFile to GUR_TIME.log
   Prev:   Default: 
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 37 columns and 135 nonzeros
Model fingerprint: 0xf6a83546
Variable types: 9 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [9e-01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 10 rows and 13 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [29]:
m.computeIIS()


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        34         -         0         9         -           0s
        2         2         -         0         0         -           0s

IIS computed: 2 constraints, 0 bounds
IIS runtime: 0.01 seconds
